In [8]:
import os

# The specific directory discovered by your search
nvcc_bin = "/home/ritesh/Downloads/ImageSRCNN/.venv/lib/python3.12/site-packages/nvidia/cuda_nvcc/bin"

# Update PATH and add the XLA fallback for extra stability
os.environ["PATH"] = nvcc_bin + os.pathsep + os.environ["PATH"]
os.environ['XLA_FLAGS'] = '--xla_gpu_unsafe_fallback_to_driver_on_ptxas_not_found=true'

import tensorflow as tf

# Verification check
import subprocess
try:
    res = subprocess.run(['ptxas', '--version'], capture_output=True, text=True)
    print("Success! ptxas is now reachable.")
    print(res.stdout.splitlines()[0])
except Exception as e:
    print("Link failed. Check if the path exists or use tf.config.optimizer.set_jit(False)")

Success! ptxas is now reachable.
ptxas: NVIDIA (R) Ptx optimizing assembler


**Import Libraries**

In [9]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os

**Configuration**

In [10]:
BATCH_SIZE = 2
IMAGE_SIZE = 256  # Target High Resolution Size
DOWNSCALE_FACTOR = 4
EPOCHS = 1
CHANNELS = 3
# Calculate Low Resolution Size
LR_SIZE = IMAGE_SIZE // DOWNSCALE_FACTOR

**Data Preprocessing**

In [11]:
def process_input(image):
    image = image / 255.0

    img_lr = tf.image.resize(image, (LR_SIZE, LR_SIZE), method="bicubic")

    return img_lr, image # Input (LR), Target (HR)

raw_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/home/ritesh/Downloads/ImageSRCNN/trainning/data/train/DIV2K_HR",
    label_mode=None,
    seed=123,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

# Map the processing function
train_ds = raw_dataset.map(process_input).prefetch(buffer_size=tf.data.AUTOTUNE)

Found 800 files.


**Build the Super Resolution CNN (SRCNN)**

In [12]:
def get_sr_model(upscale_factor=4, channels=3):
    inputs = layers.Input(shape=(None, None, channels))

    # 1. Upsampling using Bicubic Interpolation to match target size
    # (Alternatively, you can use Conv2DTranspose for learnable upsampling)
    x = layers.UpSampling2D(size=(upscale_factor, upscale_factor), interpolation='bilinear')(inputs)

    # 2. Feature Extraction (Patch extraction and representation)
    x = layers.Conv2D(64, (9, 9), padding='same', activation='relu')(x)

    # 3. Non-linear mapping
    x = layers.Conv2D(32, (5, 5), padding='same', activation='relu')(x)

    # 4. Reconstruction
    outputs = layers.Conv2D(channels, (5, 5), padding='same', activation='linear')(x)

    model = models.Model(inputs, outputs, name="SRCNN")
    return model

model = get_sr_model(upscale_factor=DOWNSCALE_FACTOR)
model.summary()

Model: "SRCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, None, None, 64) │        15,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, None, None, 32) │        51,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, None, None, 3)  │         2,403 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,251 (270.51 KB)

 Trainable params: 69,251 (270.51 KB)

 Non-trainable params: 0 (0.00 B)

**Training**

In [13]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

# Train
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    verbose=1
)

  7/400 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - loss: 0.1525 - mean_absolute_error: 0.2921

I0000 00:00:1770319823.159916   26122 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - loss: 0.0188 - mean_absolute_error: 0.0908


**Visualization & Comparison (Results)**

In [15]:
def plot_results(model, dataset, num_samples=3):
    # Take a single batch
    for lr_batch, hr_batch in dataset.take(1):
        
        # Check how many images are actually in this batch
        batch_size = lr_batch.shape[0]
        # Don't try to plot more than we actually have
        actual_samples = min(num_samples, batch_size)
        
        plt.figure(figsize=(15, 5 * actual_samples))
        
        # Generate Super Resolution images
        sr_batch = model.predict(lr_batch)
        
        for i in range(actual_samples):
            # ... (rest of your plotting code)

SyntaxError: incomplete input (3805647479.py, line 16)